In [1]:
#imports of libraries for webscraping understat
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

In [2]:
# check match's soup
url_match = 'https://understat.com/match/5186'
response = requests.get(url_match)
soup_match = BeautifulSoup(response.text, 'html.parser')
soup_match

<!DOCTYPE html>

<html>
<head>
<base href="https://understat.com/"/>
<title>FC Cologne 2 - 2 Wolfsburg (May 23 2015) | Bundesliga | 2014/2015 | xG | Understat.com</title>
<meta charset="utf-8"/>
<meta content="FC Cologne 2 - 2 Wolfsburg. Check out detailed player statistic, goals, assists, key passes, xG, shot map, xGplot." name="description"/>
<meta content="FC Cologne, Wolfsburg, Bundesliga, 2014/2015, (May 23 2015), xG, expected goals, shot map" name="Keywords"/>
<link href="apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
<link href="favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
<link href="manifest.json" rel="manifest"/>
<link color="#5bbad5" href="safari-pinned-tab.svg" rel="mask-icon"/>
<meta content="understat" name="apple-mobile-web-app-title"/>
<meta content="understat" name="application-name"/>
<meta content="#ffffff" name="theme-color"/>
<meta content="no-cache" http-eq

In [3]:
#to check
#soup_match.find('div', class_='progress-bar').parent

In [12]:
#define function to get one match's stat
def get_match_stats(match_id):
    #get soup for match match_id
    url_match = 'https://understat.com/match/' + match_id
    response = requests.get(url_match)
    soup_match = BeautifulSoup(response.text, 'html.parser')

    #Add date to dictionary first
    breadcrumb = soup_match.find('ul', class_='breadcrumb')
    # Locate the date, which is the last <li> element in the breadcrumb
    date_li = breadcrumb.find_all('li')[-1]
    match_date = date_li.text.strip()
    # Add the date to the match stats dictionary
    match_stats = {'Date': match_date}

    # Extract team names and add to match_stats dict
    teams_section = soup_match.find('div', class_='progress-bar teams-titles')
    home_team = teams_section.find('div', class_='progress-home').text.strip()
    away_team = teams_section.find('div', class_='progress-away').text.strip()
    match_stats['Home Team'] = home_team 
    match_stats['Away Team'] = away_team
    
    #list of required stats and loop through them
    stats_titles = ['xG', 'DEEP', 'PPDA', 'xPTS']
    for stat_title in stats_titles:
        # Find the progress bar whose progress-title matches the stat_title
        stat_section = soup_match.find('div', class_='progress-title', string=stat_title)
        if stat_section:
            # Navigate to the parent progress-bar div
            parent = stat_section.find_parent('div', class_='progress-bar')
                
            # Find all progress-value divs within this section
            progress_values = parent.find_all('div', class_='progress-value')
                
            # Extract the text from the first and second progress-value divs
            home_value = progress_values[0].text.strip().replace('\n', '')
            away_value = progress_values[1].text.strip().replace('\n', '') if len(progress_values) > 1 else None
            match_stats[f'{stat_title}_home'] = home_value
            match_stats[f'{stat_title}_away'] = away_value

    return match_stats

get_match_stats('5451')

{'Date': 'Aug 23 2014',
 'Home Team': 'Eintracht Frankfurt',
 'Away Team': 'Freiburg',
 'xG_home': '1.75',
 'xG_away': '1.38',
 'DEEP_home': '3',
 'DEEP_away': '13',
 'PPDA_home': '10.52',
 'PPDA_away': '3.47',
 'xPTS_home': '1.65',
 'xPTS_away': '1.10'}

In [14]:
url_league = 'https://understat.com/league/Bundesliga/2014'
response_league = requests.get(url_league)
soup_league = BeautifulSoup(response_league.text, 'html.parser')

In [15]:
year_list = range(2014,2025)
match_ids = []
for year in year_list:
    url_league = 'https://understat.com/league/Bundesliga/' + str(year)
    response_league = requests.get(url_league)
    soup_league = BeautifulSoup(response_league.text, 'html.parser')
    
    # Find the script tag containing the 'datesData' variable
    script = soup_league.find('script', text=lambda t: t and 'datesData' in t).string
    
    # Extract the JSON part from the script
    start_index = script.find("JSON.parse('") + len("JSON.parse('")
    end_index = script.find("')", start_index)
    json_data = script[start_index:end_index].encode('utf-8').decode('unicode_escape')
    
    # Parse the JSON data
    matches_data = json.loads(json_data)
    
    # Extract and add all match IDs to match id list
    for match in matches_data:
        match_ids.append(match['id'])
    
# Print the list of match IDs to check
print(match_ids)

/var/folders/72/b7zxktp96cz1n4tjk3mlbj5w0000gn/T/ipykernel_3553/1347370434.py:9: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  script = soup_league.find('script', text=lambda t: t and 'datesData' in t).string


['5447', '5448', '5449', '5450', '5451', '5452', '5453', '5454', '5455', '5456', '5457', '5458', '5459', '5460', '5461', '5462', '5463', '5464', '5411', '5412', '5413', '5414', '5415', '5416', '5417', '5418', '5419', '5420', '5421', '5422', '5423', '5424', '5425', '5426', '5427', '5428', '5429', '5430', '5431', '5432', '5433', '5434', '5435', '5436', '5437', '5438', '5439', '5440', '5441', '5442', '5443', '5444', '5445', '5446', '5383', '5384', '5385', '5386', '5387', '5388', '5389', '5390', '5391', '5392', '5393', '5394', '5395', '5396', '5397', '5398', '5399', '5400', '5401', '5402', '5403', '5404', '5405', '5406', '5407', '5408', '5409', '5410', '5348', '5349', '5350', '5351', '5352', '5353', '5354', '5355', '5356', '5357', '5358', '5359', '5360', '5361', '5362', '5363', '5364', '5365', '5366', '5367', '5368', '5369', '5370', '5371', '5372', '5373', '5374', '5375', '5376', '5377', '5378', '5379', '5380', '5381', '5382', '5312', '5313', '5314', '5315', '5316', '5317', '5318', '5319',

In [18]:
len(match_ids)
# -> 3366, also 11 saisons , weil /2024 die aktuelle saison ist und schon alle match_ids angelegt sind, 
#auch wenn die matches noch niht gespielt sind und die understat seite noch einen 404 fehler zeigt

3366

In [30]:
buli_data = []

In [32]:
for match_id in match_ids:
    buli_data.append(get_match_stats(match_id))

AttributeError: 'NoneType' object has no attribute 'find_all'

In [36]:
len(buli_data)

3195

In [38]:
buli_data

[{'Date': 'Aug 22 2014',
  'Home Team': 'Bayern Munich',
  'Away Team': 'Wolfsburg',
  'xG_home': '2.57',
  'xG_away': '1.20',
  'DEEP_home': '5',
  'DEEP_away': '4',
  'PPDA_home': '9.63',
  'PPDA_away': '21.85',
  'xPTS_home': '2.35',
  'xPTS_away': '0.47'},
 {'Date': 'Aug 23 2014',
  'Home Team': 'Hoffenheim',
  'Away Team': 'Augsburg',
  'xG_home': '1.53',
  'xG_away': '0.28',
  'DEEP_home': '6',
  'DEEP_away': '2',
  'PPDA_home': '6.59',
  'PPDA_away': '5.74',
  'xPTS_home': '2.51',
  'xPTS_away': '0.31'},
 {'Date': 'Aug 23 2014',
  'Home Team': 'Hannover 96',
  'Away Team': 'Schalke 04',
  'xG_home': '1.18',
  'xG_away': '0.96',
  'DEEP_home': '4',
  'DEEP_away': '3',
  'PPDA_home': '9.64',
  'PPDA_away': '6.06',
  'xPTS_home': '1.57',
  'xPTS_away': '1.12'},
 {'Date': 'Aug 23 2014',
  'Home Team': 'Hertha Berlin',
  'Away Team': 'Werder Bremen',
  'xG_home': '1.76',
  'xG_away': '1.19',
  'DEEP_home': '3',
  'DEEP_away': '5',
  'PPDA_home': '5.69',
  'PPDA_away': '9.87',
  'xPTS

In [40]:
df = pd.DataFrame(buli_data)
df

,Date,Home Team,Away Team,xG_home,xG_away,DEEP_home,DEEP_away,PPDA_home,PPDA_away,xPTS_home,xPTS_away
0,Aug 22 2014,Bayern Munich,Wolfsburg,2.57,1.20,5,4,9.63,21.85,2.35,0.47
1,Aug 23 2014,Hoffenheim,Augsburg,1.53,0.28,6,2,6.59,5.74,2.51,0.31
2,Aug 23 2014,Hannover 96,Schalke 04,1.18,0.96,4,3,9.64,6.06,1.57,1.12
3,Aug 23 2014,Hertha Berlin,Werder Bremen,1.76,1.19,3,5,5.69,9.87,1.83,0.85
4,Aug 23 2014,Eintracht Frankfurt,Freiburg,1.75,1.38,3,13,10.52,3.47,1.65,1.10
...,...,...,...,...,...,...,...,...,...,...,...
3190,Dec 21 2024,Werder Bremen,Union Berlin,2.73,1.64,11,14,9.48,12.88,2.14,0.66
3191,Dec 21 2024,Holstein Kiel,Augsburg,2.49,1.13,10,9,13.86,7.67,2.38,0.46
3192,Dec 21 2024,Bayer Leverkusen,Freiburg,3.26,1.34,14,4,6.76,23.88,2.55,0.33
3193,Dec 22 2024,Bochum,FC Heidenheim,1.68,0.51,9,12,6.84,7.46,2.25,0.52


In [42]:
csv_file_path = "../data/other_stats/adv_stats.csv"  # Specify your desired file path
df.to_csv(csv_file_path, index=False)